# Import

In [52]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import numpy as np

# Hyperparameter

# Load Data

In [35]:
#augmented, 500개로 맞춤
#augmented2, 1000개로 맞춤
#augmented_combined, 동일 카테고리별로 5개씩 묶고, 합친 갯수가 200개보다 적으면 200개까지 증강
#augmented_combined3, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined4, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#augmented_combined5, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 200개보다 적으면 50개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined6, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 7000개까지 줄임
#augmented_combined8, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 5000개까지 줄임
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 3000개까지 줄임
#augmented_combined7, 셔플 추가, 동일 카테고리별로 2개씩 묶고, 합친 갯수가 30개보다 적으면 30개까지 증강
#그리고 지역은 2000개까지 줄임
train_df = pd.read_csv("/kaggle/input/dacon-dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/dacon-dataset/test.csv")

In [36]:
train_df["분류"].value_counts()

분류
지역               26950
경제:부동산            3454
사회:사건_사고          2568
경제:반도체            2318
사회:사회일반           1480
사회:교육_시험           995
정치:국회_정당           966
사회:의료_건강           950
경제:취업_창업           845
스포츠:올림픽_아시안게임      841
경제:산업_기업           711
문화:전시_공연           671
경제:자동차             640
경제:경제일반            625
사회:장애인             621
스포츠:골프             617
정치:선거              608
경제:유통              589
IT_과학:모바일          537
사회:여성              536
사회:노동_복지           447
사회:환경              396
경제:서비스_쇼핑          387
경제:무역              375
정치:행정_자치           349
국제                 337
문화:방송_연예           335
스포츠:축구             328
경제:금융_재테크          327
정치:청와대             279
문화:출판              248
IT_과학:IT_과학일반      243
IT_과학:인터넷_SNS      238
문화:미술_건축           229
정치:정치일반            221
IT_과학:과학           215
문화:문화일반            213
문화:학술_문화재          202
문화:요리_여행           190
경제:자원              178
문화:종교              173
IT_과학:콘텐츠          160
사회:미디어             128
사회:날씨   

#중복만 날림

In [37]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import random

# # 1. '지역' 카테고리 필터링
# region_df = train_df[train_df['분류'] == '지역'].copy()

# # 2. 제목과 키워드 데이터를 각각 리스트로 변환
# title_texts = region_df['제목'].tolist()
# keyword_texts = region_df['키워드'].tolist()

# # 3. TF-IDF 벡터화
# title_vectorizer = TfidfVectorizer().fit_transform(title_texts)
# keyword_vectorizer = TfidfVectorizer().fit_transform(keyword_texts)

# # 4. 코사인 유사도 계산 (제목과 키워드 각각)
# title_cosine_similarities = cosine_similarity(title_vectorizer, title_vectorizer)
# keyword_cosine_similarities = cosine_similarity(keyword_vectorizer, keyword_vectorizer)

# # 5. 유사도 임계값 설정 (0.8 이상이면 중복으로 간주)
# threshold = 0.5

# # 6. 중복 데이터 탐지 함수 (일부 제거)
# def find_duplicate_indices(cosine_similarities, threshold):
#     duplicate_indices = []
#     for i in range(len(cosine_similarities)):
#         for j in range(i + 1, len(cosine_similarities)):
#             if cosine_similarities[i][j] > threshold:
#                 duplicate_indices.append(j)
#     # 중복된 데이터 중에서 절반만 제거하도록 선택
#     unique_duplicate_indices = list(set(duplicate_indices))
#     to_remove = random.sample(unique_duplicate_indices, len(unique_duplicate_indices) // 2)
#     return to_remove

# # 7. 제목과 키워드에서 각각 유사한 데이터의 중복 인덱스 찾기
# title_duplicate_indices = find_duplicate_indices(title_cosine_similarities, threshold)
# keyword_duplicate_indices = find_duplicate_indices(keyword_cosine_similarities, threshold)

# # 8. 제목과 키워드 중복 인덱스의 교집합만 제거
# duplicate_indices = list(set(title_duplicate_indices) | set(keyword_duplicate_indices)) 
# # 9. 중복 데이터 제거
# region_df_cleaned = region_df.drop(region_df.index[duplicate_indices]).reset_index(drop=True)


# # 10. 중복된 데이터 수 확인
# print(f"지역 카테고리에서 중복된 데이터 {len(duplicate_indices)}개 제거 완료.")

In [38]:
# 11. 원래 데이터프레임에서 지역 카테고리를 업데이트
# train_df_updated = pd.concat([train_df[train_df['분류'] != '지역'], region_df_cleaned], ignore_index=True)

# 제거된 후의 데이터프레임 확인
#print(f"전체 데이터 크기: {train_df_updated.shape}")
# print(f"전체 데이터 크기: {train_df.shape}")

In [39]:
#train_df_updated["분류"].value_counts()

#비슷한거 날리고 2개 합침（새로운 형태의 데이터가 필요하면 변경）


In [40]:
def combine_articles(df):
    combined_data = []
    removed_count = 0  # 삭제된 데이터 수를 추적
    threshold = 0.7  # 유사도 임계값

    for category in df['분류'].unique():
        category_data = df[df['분류'] == category].reset_index(drop=True)

        # 카테고리 데이터가 10,000개 이상일 경우에만 진행, 지역만 적용됨
        if len(category_data) > 10000:

            # 제목과 키워드를 합친 텍스트 데이터 생성
            category_data['combined_text'] = category_data['제목'] + " " + category_data['키워드']

            # TF-IDF 벡터화
            vectorizer = TfidfVectorizer()
            tfidf_matrix = vectorizer.fit_transform(category_data['combined_text'])

            # 코사인 유사도 계산
            cosine_sim = cosine_similarity(tfidf_matrix)

            # 유사한 기사 2개씩 묶기 + 일부 중복 제거
            used_indices = set()

            # 중복 제거를 위해 일정 임계값 이상의 유사도를 가진 데이터 탐색
            duplicate_indices = []
            for i in range(len(cosine_sim)):
                for j in range(i + 1, len(cosine_sim)):
                    if cosine_sim[i][j] > threshold:
                        duplicate_indices.append(j)

            # 중복된 데이터의 절반만 제거
            unique_duplicate_indices = list(set(duplicate_indices))
            to_remove = random.sample(unique_duplicate_indices, len(unique_duplicate_indices) // 2)
            removed_count += len(to_remove)  # 삭제된 데이터 수 기록

            # 중복 제거 후, 인덱스가 올바른지 확인
            for i in range(len(category_data)):
                if i in used_indices or i in to_remove:
                    continue
                
                # 가장 유사한 기사 선택
                most_similar_idx = np.argsort(cosine_sim[i])[-2]  # 자기 자신을 제외한 가장 유사한 기사 선택
                
                # most_similar_idx가 사용되었거나 제거된 항목인지 확인
                if most_similar_idx in used_indices or most_similar_idx in to_remove:
                    continue
                
                combined_title = category_data['제목'][i] + " " + category_data['제목'][most_similar_idx]
                combined_keywords = category_data['키워드'][i] + "," + category_data['키워드'][most_similar_idx]
                
                combined_entry = {
                    '분류': category,
                    '제목': combined_title,
                    '키워드': combined_keywords
                }
                
                combined_data.append(combined_entry)
                used_indices.add(i)
                used_indices.add(most_similar_idx)

        else:
            # 데이터가 10,000개 이하인 경우 2개씩 단순 묶기
            for i in range(0, len(category_data), 1):
                combined_title = " ".join(category_data['제목'][i:i+2])
                combined_keywords = ",".join(category_data['키워드'][i:i+2])
                combined_entry = {
                    '분류': category,
                    '제목': combined_title,
                    '키워드': combined_keywords
                }
                combined_data.append(combined_entry)

    # 총 몇 개의 데이터가 합병되었는지 확인 출력
    print(f"총 {removed_count}개의 데이터가 유사해서 합병 또는 제거되었습니다.")
    
    combined_df = pd.DataFrame(combined_data)

    # CSV로 저장
    combined_df.to_csv("/kaggle/working/editted.csv", index=False, encoding='utf-8-sig')
    print("CSV 파일로 저장되었습니다: editted.csv")
    
    return combined_df

# 사용 예시
# df = pd.DataFrame({'분류': [...], '제목': [...], '키워드': [...]})
# new_df = combine_articles(df)


In [41]:
combined_df = combine_articles(train_df)

총 6492개의 데이터가 유사해서 합병 또는 제거되었습니다.
CSV 파일로 저장되었습니다: editted.csv


In [42]:
final_df=combined_df

In [43]:
# 줄인 데이터 확인
print(final_df['분류'].value_counts())

분류
지역               6289
경제:부동산           3454
사회:사건_사고         2568
경제:반도체           2318
사회:사회일반          1480
사회:교육_시험          995
정치:국회_정당          966
사회:의료_건강          950
경제:취업_창업          845
스포츠:올림픽_아시안게임     841
경제:산업_기업          711
문화:전시_공연          671
경제:자동차            640
경제:경제일반           625
사회:장애인            621
스포츠:골프            617
정치:선거             608
경제:유통             589
IT_과학:모바일         537
사회:여성             536
사회:노동_복지          447
사회:환경             396
경제:서비스_쇼핑         387
경제:무역             375
정치:행정_자치          349
국제                337
문화:방송_연예          335
스포츠:축구            328
경제:금융_재테크         327
정치:청와대            279
문화:출판             248
IT_과학:IT_과학일반     243
IT_과학:인터넷_SNS     238
문화:미술_건축          229
정치:정치일반           221
IT_과학:과학          215
문화:문화일반           213
문화:학술_문화재         202
문화:요리_여행          190
경제:자원             178
문화:종교             173
IT_과학:콘텐츠         160
사회:미디어            128
사회:날씨             124
스포츠:농구_배구         113
문화:음악  

# Custom Dataset

In [44]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

In [45]:
# # 데이터 준비
# train_df['제목_키워드'] = train_df['제목'] + ' ' + train_df['키워드']
# test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# # 레이블 인코딩
# label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
# train_df['label'] = train_df['분류'].map(label_encoder)

# # 데이터 분할 (train -> train + validation)
# train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# # 데이터셋 생성
# train_dataset = TextDataset(train_df.제목_키워드.tolist(), train_df.label.tolist(), tokenizer)
# val_dataset = TextDataset(val_df.제목_키워드.tolist(), val_df.label.tolist(), tokenizer)
# test_dataset = TextDataset(test_df.제목_키워드.tolist(), None, tokenizer)  # 라벨 없음

# # 데이터 로더 생성
# train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [46]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
# #tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
# model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

In [47]:
# 데이터 준비
final_df['제목_키워드'] = final_df['제목'] + ' ' + final_df['키워드']
test_df['제목_키워드'] = test_df['제목'] + ' ' + test_df['키워드']

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(final_df['분류'].unique())}
final_df['label'] = final_df['분류'].map(label_encoder)

# 데이터 분할 (final_df -> train + validation)
train_df, val_df = train_test_split(final_df, test_size=0.2, stratify=final_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.제목_키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.제목_키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.제목_키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [48]:
# 옵티마이저 및 학습 파라미터 설정
#optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

In [49]:
# # 학습
# model.train()
# #for epoch in range(CFG.epoch):
# for epoch in range(CFG.epoch):
#     for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

#     # Validation
#     model.eval()
#     val_predictions = []
#     val_true_labels = []
#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc='Validating'):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             _, preds = torch.max(outputs.logits, dim=1)
#             val_predictions.extend(preds.cpu().tolist())
#             val_true_labels.extend(labels.cpu().tolist())
    
#     # 검증 결과 출력
#     val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
#     print(f"Validation F1 Score: {val_f1:.2f}")

In [50]:
config = {
    "learning_rate": 0.0001,   # 학습률
    "epoch": 15,               # 에폭 수
    "batch_size": 64,          # 배치 크기
    "momentum": 0.9,           # 모멘텀
    "weight_decay": 0.0005,    # 가중치 감쇠
    "dropout_rate": 0.3,       # 드롭아웃 비율
    "gradient_clipping": 1.0   # 그래디언트 클리핑
}

CFG = SimpleNamespace(**config)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 토크나이저 초기화 (BertTokenizer 사용)
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# BERT 구성 설정에서 드롭아웃 비율 변경 (예: 0.3으로 설정)
config = BertConfig.from_pretrained('monologg/kobert', hidden_dropout_prob=0.3, num_labels=len(train_df['분류'].unique()))

# 모델 초기화 (드롭아웃 비율 적용)
model = BertForSequenceClassification.from_pretrained(
    'monologg/kobert', 
    config=config
).to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score

# 모델 학습 함수
def train_and_validate(model, train_loader, val_loader, optimizer, scheduler, CFG, device):
    model.train()
    best_val_f1 = 0
    patience = 3
    early_stopping_counter = 0

    for epoch in range(CFG.epoch):
        # Training
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            
            # Gradient Clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.gradient_clipping)
            
            optimizer.step()
            total_loss += loss.item()
        
        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{CFG.epoch}, Training Loss: {avg_train_loss:.4f}")
        
        # Validation
        model.eval()
        val_predictions = []
        val_true_labels = []
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validating'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()
                _, preds = torch.max(outputs.logits, dim=1)
                val_predictions.extend(preds.cpu().tolist())
                val_true_labels.extend(labels.cpu().tolist())
        
        avg_val_loss = val_loss / len(val_loader)
        val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
        
        print(f"Validation Loss: {avg_val_loss:.4f}, Validation F1 Score: {val_f1:.2f}")
        
        # Early Stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("Early stopping triggered!")
                break
        
        # Learning Rate Scheduler step
        scheduler.step()


In [ ]:
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate, weight_decay=CFG.weight_decay)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # 매 5 에폭마다 학습률을 10% 감소

# Train and validate (train_loader와 val_loader가 이미 정의되어 있다고 가정)
train_and_validate(model, train_loader, val_loader, optimizer, scheduler, CFG, device)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/15:   4%|▍         | 19/425 [00:26<09:02,  1.34s/it]

# Inference

In [ ]:
model.eval()
test_predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        # 모델을 통한 예측
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        
        # 예측 결과 저장
        test_predictions.extend(preds.cpu().tolist())

# test.csv 파일에서 'id' 값을 그대로 사용
test_df = pd.read_csv('/kaggle/input/dacon-dataset/test.csv')  # test.csv에 'id' 칼럼이 존재한다고 가정

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]



In [ ]:
# 예측 결과를 데이터프레임으로 저장
df_results = pd.DataFrame({
    'ID': test_df['ID'],             # test.csv의 'id' 값을 그대로 사용
    '분류': decoded_predictions  # 디코딩된 예측 라벨
})

# CSV 파일로 저장
df_results.to_csv('/kaggle/working/submission.csv', index=False)

print("예측 결과가 submission.csv 파일로 저장되었습니다.")

In [ ]:
df=pd.read_csv('/kaggle/working/submission.csv')

In [ ]:
df.head(50)